In [1]:
from mmcv import Config
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)

ValueError: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216 from C header, got 192 from PyObject

In [2]:
classes = ("UNKNOWN", "General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")
# config file 들고오기
cfg = Config.fromfile('./configs/scnet/scnet_r50_fpn_20e_coco.py')

PREFIX = '../../input/data/'


# dataset 바꾸기
cfg.data.train.classes = classes
cfg.data.train.img_prefix = PREFIX
cfg.data.train.ann_file = PREFIX + 'train.json'
cfg.data.train.pipeline[2]['img_scale'] = (512, 512)
cfg.data.train.pipeline[1].with_seg = False

cfg.data.val.classes = classes
cfg.data.val.img_prefix = PREFIX
cfg.data.val.ann_file = PREFIX + 'val.json'
cfg.data.val.pipeline[1]['img_scale'] = (512, 512)

cfg.data.test.classes = classes
cfg.data.test.img_prefix = PREFIX
cfg.data.test.ann_file = PREFIX + 'test.json'
cfg.data.test.pipeline[1]['img_scale'] = (512, 512)

cfg.data.samples_per_gpu = 4
cfg.data.workers_per_gpu = 4
cfg.seed=90
cfg.gpu_ids = [0]
cfg.work_dir = './work_dirs/scnet_r50_fpn_20e_coco'

#cfg.model.roi_head.bbo.
cfg.model.roi_head.bbox_head[0].num_classes = 11
cfg.model.roi_head.bbox_head[1].num_classes = 11
cfg.model.roi_head.bbox_head[2].num_classes = 11
cfg.model.roi_head.mask_head.num_classes = 11
cfg.model.roi_head.glbctx_head.num_classes = 11
#cfg.model.roi_head.semantic_head.num_classes = 11
#cfg.runner.max_epochs = 200
#cfg.optimizer = dict(type='Adam', lr=1e-4, weight_decay=0.0001)
cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.evaluation.metric = ['bbox']
cfg.checkpoint_config['max_keep_ckpts']=5
cfg.evaluation.save_best = 'bbox_mAP_50'
cfg.log_config.hooks = [dict(type='TextLoggerHook'), dict(type='WandbLoggerHook')]

In [3]:
cfg.lr_config['policy'] = 'Step'

In [4]:
print(cfg.evaluation.metric)

['bbox']


In [5]:
for key in cfg.data.val:
    print(key)

type
ann_file
img_prefix
pipeline
classes


In [6]:
print(cfg.data.val)

{'type': 'CocoDataset', 'ann_file': '../../input/data/val.json', 'img_prefix': '../../input/data/', 'pipeline': [{'type': 'LoadImageFromFile'}, {'type': 'MultiScaleFlipAug', 'img_scale': (512, 512), 'flip': False, 'transforms': [{'type': 'Resize', 'keep_ratio': True}, {'type': 'RandomFlip', 'flip_ratio': 0.5}, {'type': 'Normalize', 'mean': [123.675, 116.28, 103.53], 'std': [58.395, 57.12, 57.375], 'to_rgb': True}, {'type': 'Pad', 'size_divisor': 32}, {'type': 'ImageToTensor', 'keys': ['img']}, {'type': 'Collect', 'keys': ['img']}]}], 'classes': ('UNKNOWN', 'General trash', 'Paper', 'Paper pack', 'Metal', 'Glass', 'Plastic', 'Styrofoam', 'Plastic bag', 'Battery', 'Clothing')}


In [7]:
print(cfg.data.train.pipeline[8]['keys'].remove('gt_semantic_seg'))

None


In [8]:
model = build_detector(cfg.model)

2021-05-13 00:43:37,148 - mmdet - INFO - load model from: torchvision://resnet50
2021-05-13 00:43:37,149 - mmdet - INFO - Use load_from_torchvision loader
2021-05-13 00:43:37,406 - mmdet - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: fc.weight, fc.bias



In [9]:
datasets = [build_dataset(cfg.data.train)]

loading annotations into memory...
Done (t=3.52s)
creating index...
index created!


In [10]:
train_detector(model, datasets[0], cfg, distributed=False, validate=True)

loading annotations into memory...


2021-05-13 00:43:46,362 - mmdet - INFO - Start running, host: root@bc720df3e041, work_dir: /opt/ml/code/mmdetection_trash/work_dirs/scnet_r50_fpn_20e_coco
2021-05-13 00:43:46,363 - mmdet - INFO - workflow: [('train', 1)], max: 20 epochs


Done (t=0.87s)
creating index...
index created!


/opt/ml/code/mmdetection_trash/mmdet/core/evaluation/eval_hooks.py:111: UserWarning: runner.meta is None. Creating a empty one.
  warnings.warn('runner.meta is None. Creating a empty one.')
wandb: Currently logged in as: k1smet14 (use `wandb login --relogin` to force relogin)


2021-05-13 00:44:26,027 - mmdet - INFO - Epoch [1][50/655]	lr: 1.978e-03, eta: 2:33:28, time: 0.706, data_time: 0.073, memory: 3108, loss_rpn_cls: 0.6329, loss_rpn_bbox: 0.0657, loss_glbctx: 1.7652, s0.loss_cls: 0.8259, s0.acc: 87.5693, s0.loss_bbox: 0.1441, s1.loss_cls: 0.4300, s1.acc: 81.5039, s1.loss_bbox: 0.0420, s2.loss_cls: 0.2354, s2.acc: 80.9102, s2.loss_bbox: 0.0057, loss_mask: 1.9647, loss: 6.1116, grad_norm: 27.0128
2021-05-13 00:45:00,210 - mmdet - INFO - Epoch [1][100/655]	lr: 3.976e-03, eta: 2:30:30, time: 0.684, data_time: 0.022, memory: 3261, loss_rpn_cls: 0.2833, loss_rpn_bbox: 0.0648, loss_glbctx: 1.2538, s0.loss_cls: 0.4601, s0.acc: 90.2764, s0.loss_bbox: 0.2266, s1.loss_cls: 0.1299, s1.acc: 95.6807, s1.loss_bbox: 0.0655, s2.loss_cls: 0.0414, s2.acc: 97.7139, s2.loss_bbox: 0.0090, loss_mask: 1.2769, loss: 3.8111, grad_norm: 11.6381
2021-05-13 00:47:14,848 - mmdet - INFO - Epoch [1][300/655]	lr: 1.197e-02, eta: 2:25:08, time: 0.676, data_time: 0.023, memory: 3261, los

...

Loading and preparing results...
DONE (t=0.80s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=8.15s).
Accumulating evaluation results...


2021-05-13 04:51:20,073 - mmdet - INFO - Now best checkpoint is epoch_20.pth.Best bbox_mAP_50 is 0.2690
2021-05-13 04:51:20,113 - mmdet - INFO - Epoch(val) [20][655]	bbox_mAP: 0.1670, bbox_mAP_50: 0.2690, bbox_mAP_75: 0.1660, bbox_mAP_s: 0.0330, bbox_mAP_m: 0.1450, bbox_mAP_l: 0.3550, bbox_mAP_copypaste: 0.167 0.269 0.166 0.033 0.145 0.355


DONE (t=1.70s).
Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.167
Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.269
Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.166
Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.033
Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.145
Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.355
Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.381
Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=300 ] = 0.381
Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=1000 ] = 0.381
Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.125
Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.350
Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.619


train/loss_rpn_cls,0.04879
train/loss_rpn_bbox,0.03967
train/loss_glbctx,0.50962
train/s0.loss_cls,0.2397
train/s0.acc,92.13574
train/s0.loss_bbox,0.12739
train/s1.loss_cls,0.10274
train/s1.acc,93.35214
train/s1.loss_bbox,0.1039
train/s2.loss_cls,0.05198
train/s2.acc,93.32516


train/loss_rpn_cls,█▆▄▄▄▄▂▃▃▃▂▃▃▂▃▃▂▂▂▂▃▃▂▄▂▃▃▃▃▄▂▂▁▁▁▁▁▁▁▁
train/loss_rpn_bbox,█▇▅▆▅▄▃▅▄▄▄▅▄▃▄▃▃▃▂▃▄▄▄▅▄▅▄▅▃▅▄▃▂▂▁▁▁▂▂▃
train/loss_glbctx,██▇▇▇▆▇▆▆▆▆▆▆▆▆▅▅▆▅▅▆▆▅▆▅▅▅▅▅▆▅▄▄▃▂▂▂▂▁▁
train/s0.loss_cls,█▇▅▄▄▃▅▄▄▄▃▅▃▃▃▃▃▃▂▃▃▃▃▃▃▃▃▃▃▃▃▂▂▁▁▁▁▁▁▁
train/s0.acc,▃▁▂▃▃▄▂▄▄▄▅▂▄▄▅▆▄▅▅▄▄▄▅▅▄▄▅▄▅▄▄▇▇▇█▇█▇██
train/s0.loss_bbox,█▇▅▄▄▃▄▃▃▃▂▄▃▃▂▂▃▂▂▃▃▂▂▃▃▃▂▂▂▄▃▁▂▁▁▂▁▂▁▂
train/s1.loss_cls,▄██▆▇▅▇▆▆▆▅▇▅▅▅▄▅▅▄▅▅▅▄▅▅▅▄▄▄▅▅▃▂▁▁▁▁▁▁▁
train/s1.acc,█▃▁▂▂▃▁▃▂▂▃▁▃▃▃▄▃▃▃▂▃▃▃▃▃▃▄▃▄▃▃▅▅▆▆▆▆▆▆▆
train/s1.loss_bbox,▁██▇▇▆▇▆▆▇▅▇▅▆▅▄▆▅▅▆▅▅▅▅▆▅▅▅▄▆▆▄▄▃▃▄▃▄▃▃
train/s2.loss_cls,▁▅█▇▇▆█▆▇▇▆█▆▆▆▅▇▆▅▇▆▆▆▆▆▆▅▆▅▆▆▄▄▃▃▃▃▃▃▃
train/s2.acc,█▆▂▂▂▂▁▂▂▂▂▁▂▂▂▃▂▂▂▁▂▂▂▂▂▂▃▂▃▂▂▃▃▄▄▄▄▄▄▄


In [12]:
import mmcv
from mmcv import Config
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)
from mmdet.models import build_detector
from mmdet.apis import single_gpu_test
from mmcv.runner import load_checkpoint
import os
from mmcv.parallel import MMDataParallel
import pandas as pd
from pandas import DataFrame
from pycocotools.coco import COCO
import numpy as np

In [20]:
classes = ("UNKNOWN", "General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")
# config file 들고오기
cfg = Config.fromfile('./configs/scnet/scnet_r50_fpn_20e_coco.py')

PREFIX = '../../input/data/'

epoch = 24

# dataset 바꾸기
cfg.data.train.classes = classes
cfg.data.train.img_prefix = PREFIX
cfg.data.train.ann_file = PREFIX + 'train.json'
cfg.data.train.pipeline[2]['img_scale'] = (512, 512)

cfg.data.val.classes = classes
cfg.data.val.img_prefix = PREFIX
cfg.data.val.ann_file = PREFIX + 'val.json'
cfg.data.val.pipeline[1]['img_scale'] = (512, 512)

cfg.data.test.classes = classes
cfg.data.test.img_prefix = PREFIX
cfg.data.test.ann_file = PREFIX + 'test.json'
cfg.data.test.pipeline[1]['img_scale'] = (512, 512)

cfg.data.samples_per_gpu = 4

cfg.seed=90
cfg.gpu_ids = [0]
cfg.work_dir = './work_dirs/scnet_r50_fpn_20e_coco'

#cfg.model.roi_head.bbo.
cfg.model.roi_head.bbox_head[0].num_classes = 11
cfg.model.roi_head.bbox_head[1].num_classes = 11
cfg.model.roi_head.bbox_head[2].num_classes = 11
cfg.model.roi_head.mask_head.num_classes = 11
cfg.model.roi_head.glbctx_head.num_classes = 11

cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.model.train_cfg = None



In [21]:
checkpoint_path = os.path.join(cfg.work_dir, f'best_bbox_mAP_50.pth')


In [22]:
dataset = build_dataset(cfg.data.test)
data_loader = build_dataloader(
        dataset,
        samples_per_gpu=1,
        workers_per_gpu=cfg.data.workers_per_gpu,
        dist=False,
        shuffle=False)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [23]:
model = build_detector(cfg.model, test_cfg=cfg.get('test_cfg'))
checkpoint = load_checkpoint(model, checkpoint_path, map_location='cpu')

model.CLASSES = dataset.CLASSES
model = MMDataParallel(model.cuda(), device_ids=[0])

2021-05-13 06:07:02,179 - mmdet - INFO - load model from: torchvision://resnet50
2021-05-13 06:07:02,181 - mmdet - INFO - Use load_from_torchvision loader
2021-05-13 06:07:02,446 - mmdet - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: fc.weight, fc.bias



Use load_from_local loader


In [24]:
output = single_gpu_test(model, data_loader, show_score_thr=0.05)

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 837/837, 5.0 task/s, elapsed: 167s, ETA:     0s

In [39]:
prediction_strings = []
file_names = []
coco = COCO(cfg.data.test.ann_file)
imag_ids = coco.getImgIds()

class_num = 11
for i, out in enumerate(output):
    if i == 0:
        continue
    prediction_string = ''
    image_info = coco.loadImgs(coco.getImgIds(imgIds=i))[0]
    for j in range(class_num):
        for o in out[j]:
            prediction_string += str(j) + ' ' + str(o[4]) + ' ' + str(o[0]) + ' ' + str(o[1]) + ' ' + str(o[2]) + ' ' + str(o[3]) + ' '
        
    prediction_strings.append(prediction_string)
    file_names.append(image_info['file_name'])


submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names
submission.to_csv(os.path.join(cfg.work_dir, f'submission_best.csv'), index=None)
submission.head()

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
([array([], shape=(0, 5), dtype=float32), array([[2.8603513e+02, 3.4073251e+02, 3.0859335e+02, 3.7662430e+02,
        1.4311174e-01],
       [9.8372269e+01, 0.0000000e+00, 1.9373781e+02, 2.4812791e+02,
        8.4752008e-02],
       [1.3168793e+00, 2.3725449e+02, 3.0260773e+02, 4.5981259e+02,
        7.7452950e-02],
       [2.7363364e+02, 2.4935306e+02, 3.0782980e+02, 2.6421152e+02,
        6.6419445e-02],
       [3.5940195e+02, 4.2472943e+02, 3.7958987e+02, 4.3504590e+02,
        2.5201228e-02],
       [2.7864594e+02, 2.5401796e+02, 3.0540649e+02, 2.6711816e+02,
        1.8809114e-02],
       [8.8981812e+01, 0.0000000e+00, 5.0184436e+02, 2.5503300e+02,
        1.8792769e-02],
       [1.5252972e+02, 3.5222748e+02, 3.3093323e+02, 4.7390710e+02,
        1.8535491e-02],
       [7.4444618e+01, 9.5509842e+01, 1.3668068e+02, 2.0230635e+02,
        1.6469462e-02],
       [0.0000000e+00, 7.5057983e-01, 1.8913168

IndexError: index 4 is out of bounds for axis 0 with size 0